In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/split_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import MLP_Models, LSTM_Models, GRU_Models
from utils.benchmark import SplitBenchmark

In [2]:
conf = Config()
#mlp = MLP_Models()
#lstm = LSTM_Models()
gru = GRU_Models()
models = [gru.split_gru_0]

model_names = ["split_gru_0"]

"""
all_models = [mlp.split_mlp_3, mlp.split_mlp_2, mlp.split_mlp_1, 
        lstm.split_lstm_3, lstm.split_lstm_2, lstm.split_lstm_1, lstm.split_lstm_0,
        gru.split_gru_3, gru.split_gru_2, gru.split_gru_1, gru.split_gru_0]
all_model_names = ["split_mlp_3", "split_mlp_2", "split_mlp_1",
            "split_lstm_3", "split_lstm_2", "split_lstm_1", "split_lstm_0",
            "split_gru_3", "split_gru_2", "split_gru_1", "split_gru_0"]
"""

'\nall_models = [mlp.split_mlp_3, mlp.split_mlp_2, mlp.split_mlp_1, \n        lstm.split_lstm_3, lstm.split_lstm_2, lstm.split_lstm_1, lstm.split_lstm_0,\n        gru.split_gru_3, gru.split_gru_2, gru.split_gru_1, gru.split_gru_0]\nall_model_names = ["split_mlp_3", "split_mlp_2", "split_mlp_1",\n            "split_lstm_3", "split_lstm_2", "split_lstm_1", "split_lstm_0",\n            "split_gru_3", "split_gru_2", "split_gru_1", "split_gru_0"]\n'

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "lstm")

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)
"""
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "mlp")
mlp_test_dataset = NetworkDataset(X_test, y_test)
mlp_test_loader = DataLoader(mlp_test_dataset, conf.batch_size)
"""

'\nX_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "mlp")\nmlp_test_dataset = NetworkDataset(X_test, y_test)\nmlp_test_loader = DataLoader(mlp_test_dataset, conf.batch_size)\n'

In [ ]:
for i in range(len(models)):
    # set parameters
    model = models[i]
    #model.load()
    name = model_names[i]
    result_path = os.path.join(conf.benchmark_dpu, "split_model", name + ".txt")

    """
    # create dpu model feature
    features, labels = [], []
    if i > 2:
        for i in range(1000):
            data, label = next(iter(train_loader))
            feat, _ = model.model(data, split="dpu")
            features.append(feat)
            labels.append(label)
    else:
        for (data, label) in mlp_train_loader:
            feat, _ = model.model(data, split="dpu")
            features.append(feat)
            labels.append(label)
    
    features, labels = torch.cat(features, dim=0), torch.cat(labels, dim=0)
    dataset =  NetworkDataset(features, labels)
    loader = DataLoader(dataset, conf.batch_size)
    """

    # run benchmark
    benchmark = SplitBenchmark(model, test_loader, conf.batch_size, name, result_path, split="dpu")
    benchmark()

    # print and save result
    benchmark.print_result()
    benchmark.save()

Checkpoint loaded from /home/ubuntu/Network-Packet-ML-Model/checkpoint/split_model/split_gru_0.pth!


[W1219 09:12:45.187341651 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event


Benchmark - split_gru_0 model:

Memory usage (MB):
Avg memory usage: 126.651MB
Peak memory usage: 1923.750MB

Model inference latency on one batch (batch size = 32):
Avg latency: 2754.988ms
Min latency: 2640.211ms
Max latency: 3737.233ms

Model inference throughput (batch size = 32):
Throughput: 11.90 samples/sec

Model inference CPU usage (number of logical cores) during runtime:
CPU runtime: 138.92 seconds
Average CPU usage: 5.72/8 cores



: 